In [ ]:
using LatPhysBase
using LatticePhysics
using HDF5

# Sites / Bonds

In [ ]:
function saveBonds(
        bs :: Vector{B},
        fn :: AbstractString,
        group :: AbstractString = "bonds"
        ;
        append :: Bool = false
    ) where {LB,N,B<:Bond{LB,N}}
    
    # determine the mode based on if one wants to append stuff
    if append
        mode = "r+"
    else
        mode = "w"
    end
    
    # open the file in mode
    h5open(fn, mode) do ucfile
        # create the group in which the bonds are saved
        group_bonds = g_create(ucfile, group)
        # save the parameters
        attrs(group_bonds)["N"] = Int64(N)
        attrs(group_bonds)["L"] = string(LB)
        # save all from / to values (Int64 arrays)
        group_bonds["from"] = Int64[from(b) for b in bs]
        group_bonds["to"]   = Int64[to(b) for b in bs]
        # save all wraps
        if 0 < Int64(N)
            for n in 1:Int64(N)
                group_bonds["wrap_$(n)"] = Int64[wrap(b)[n] for b in bs]
            end
        end
        # save all labels
        if LB <: Number
            group_bonds["label"] = [label(b) for b in bs]
        else
            group_bonds["label"] = String[string(label(b)) for b in bs]
        end
    end
    
    # return nothing
    return nothing
end

In [ ]:
function loadBonds(
        ::Type{BI},
        fn :: AbstractString,
        group :: AbstractString = "bonds"
    ) where {LBI,NI,BI<:Union{Bond{LBI,NI},Bond}}
    
    # read attribute data
    attr_data = h5readattr(fn, group)
    # determine N and L based on this
    N  = attr_data["N"]
    LB = Meta.eval(Meta.parse(attr_data["L"]))
    
    # load all remaining data
    bd_from  = h5read(fn, group*"/from")
    bd_to    = h5read(fn, group*"/to")
    bd_label = h5read(fn, group*"/label")
    
    if N == 0
        bd_wrap = [Tuple([]) for i in 1:length(bd_from)]
    else
        bd_wrap_parts = [
            h5read(fn, group*"/wrap_"*string(j)) for j in 1:N
        ]
        bd_wrap = [Tuple([bd_wrap_parts[j][i] for j in 1:N]) for i in 1:length(bd_from)]
    end
    
    # create list of bonds
    bs = Bond{LB,N}[
        newBond(Bond{LB,N}, bd_from[i], bd_to[i], LB(bd_label[i]), bd_wrap[i])
        for i in 1:length(bd_from)
    ]
    
    # return nothing
    return bs
end

function loadBonds(
        fn :: AbstractString,
        group :: AbstractString = "bonds"
    )
    
    return loadBonds(Bond, fn, group)
end

In [ ]:
function saveSites(
        ss :: Vector{S},
        fn :: AbstractString,
        group :: AbstractString = "sites"
        ;
        append :: Bool = false
    ) where {LS,D,S<:Site{LS,D}}
    
    # determine the mode based on if one wants to append stuff
    if append
        mode = "r+"
    else
        mode = "w"
    end
    
    # open the file in mode
    h5open(fn, mode) do ucfile
        # create the group in which the bonds are saved
        group_sites = g_create(ucfile, group)
        # save the parameters
        attrs(group_sites)["D"] = Int64(D)
        attrs(group_sites)["L"] = string(LS)
        # save all Positions (D dimensions)
        if 0 < Int64(D)
            for n in 1:Int64(D)
                group_sites["point_$(n)"] = Float64[point(s)[n] for s in ss]
            end
        end
        # save all labels
        if LS <: Number
            group_sites["label"] = [label(s) for s in ss]
        else
            group_sites["label"] = String[string(label(s)) for s in ss]
        end
    end
    
    # return nothing
    return nothing
end

In [ ]:
function loadSites(
        ::Type{SI},
        fn :: AbstractString,
        group :: AbstractString = "sites"
    ) where {LSI,DI,SI<:Union{Site{LSI,DI},Site}}
    
    # read attribute data
    attr_data = h5readattr(fn, group)
    # determine D and L based on this
    D  = attr_data["D"]
    LS = Meta.eval(Meta.parse(attr_data["L"]))
    
    # load all remaining data
    st_label = h5read(fn, group*"/label")
    
    if D == 0
        st_point = [Float64[] for i in 1:length(st_label)]
    else
        st_point_parts = [
            h5read(fn, group*"/point_"*string(j)) for j in 1:D
        ]
        st_point = Vector{Float64}[Float64[st_point_parts[j][i] for j in 1:D] for i in 1:length(st_label)]
    end
    
    # create list of sites
    ss = Site{LS,D}[
        newSite(Site{LS,D}, st_point[i], LS(st_label[i]))
        for i in 1:length(st_label)
    ]
    
    # return nothing
    return ss
end

function loadSites(
        fn :: AbstractString,
        group :: AbstractString = "sites"
    )
    
    return loadSites(Site, fn, group)
end

In [ ]:
lt = getLatticeOpen(getUnitcellSquare(Symbol), 2)
saveBonds(bonds(lt), "test.h5")
bonds(lt)

In [ ]:
loadBonds("test.h5")

In [ ]:
uc = getUnitcellKagome(Symbol)
saveSites(sites(uc), "test.h5")
sites(uc)

In [ ]:
loadSites("test.h5")

# Unitcell

In [ ]:
function saveUnitcell(
        uc :: U,
        fn :: AbstractString,
        group :: AbstractString = "unitcell"
        ;
        append :: Bool = false
    ) where {LS,LB,D,N,S<:AbstractSite{LS,D},B<:AbstractBond{LB,N},U<:Unitcell{S,B}}
    
    # determine the mode based on if one wants to append stuff
    if append
        mode = "r+"
    else
        mode = "w"
    end
    
    # determine the site and bond group names
    group_sites = group*"/sites"
    group_bonds = group*"/bonds"
    
    # open the file in mode
    h5open(fn, mode) do ucfile
        # create the top level group
        group_uc = g_create(ucfile, group)
        # save into the attributes in which groups the sites and bonds are stored
        attrs(group_uc)["sites"] = group_sites
        attrs(group_uc)["bonds"] = group_bonds
        # save types into the attributes
        attrs(group_uc)["site_type"] = string(S)
        attrs(group_uc)["bond_type"] = string(B)
        # number of bravais lattice dimensions
        attrs(group_uc)["N"] = Int64(N)
        # save further attributes
        
        # write bravais lattice vectors
        for i in 1:N
            group_uc["a"*string(i)] = latticeVectors(uc)[i]
        end
    end
    
    # save sites and bonds into the file
    saveSites(sites(uc), fn, group_sites, append=true)
    saveBonds(bonds(uc), fn, group_bonds, append=true)
    
    # return nothing
    return nothing
end

In [ ]:
function loadUnitcell(
        ::Type{UC},
        fn :: AbstractString,
        group :: AbstractString = "unitcell"
    ) where {SUC<:AbstractSite,BUC<:AbstractBond,UC<:Union{Unitcell{SUC,BUC}, Unitcell}}
        
    # read the attribute data
    attr_data = h5readattr(fn, group)
    
    # determine the bravais lattice dimension
    N = attr_data["N"]
    
    # determine the site and bond group names
    group_sites = attr_data["sites"]
    group_bonds = attr_data["bonds"]
    
    # load bravais lattice vectors
    lattice_vectors = Vector{Float64}[]
    for i in 1:N
        push!(lattice_vectors, h5read(fn, group*"/a"*string(i)))
    end
    
    # load the sites
    site_list = loadSites(fn, group_sites)
    # load the bonds
    bond_list = loadBonds(fn, group_bonds)
    
    # determine site and bond type
    S = typeof(site_list[1])
    B = typeof(bond_list[1])
    
    # return new unitcell
    return newUnitcell(
        Unitcell{S,B},
        lattice_vectors,
        site_list,
        bond_list
    )
end

function loadUnitcell(
        fn :: AbstractString,
        group :: AbstractString = "unitcell"
    )
    
    return loadUnitcell(Unitcell, fn, group)
end

In [ ]:
uc = getUnitcellHoneycomb(String, Int64)
saveUnitcell(uc, "test.h5")
uc

In [ ]:
loadUnitcell("test.h5")

# Lattice

In [ ]:
function saveLattice(
        lt :: L,
        fn :: AbstractString,
        group :: AbstractString = "lattice"
        ;
        append :: Bool = false
    ) where {LS,LB,D,N,S<:AbstractSite{LS,D},B<:AbstractBond{LB,N},SU,BU,U<:AbstractUnitcell{SU,BU},L<:Lattice{S,B,U}}
    
    # determine the mode based on if one wants to append stuff
    if append
        mode = "r+"
    else
        mode = "w"
    end
    
    # determine the site and bond group names
    group_sites    = group*"/sites"
    group_bonds    = group*"/bonds"
    group_unitcell = group*"/unitcell"
    
    # open the file in mode
    h5open(fn, mode) do ltfile
        # create the top level group
        group_lt = g_create(ltfile, group)
        # save into the attributes in which groups the sites and bonds are stored
        attrs(group_lt)["sites"]    = group_sites
        attrs(group_lt)["bonds"]    = group_bonds
        attrs(group_lt)["unitcell"] = group_unitcell
        # save types into the attributes
        attrs(group_lt)["site_type"]     = string(S)
        attrs(group_lt)["bond_type"]     = string(B)
        attrs(group_lt)["unitcell_type"] = string(U)
        # number of bravais lattice dimensions
        attrs(group_lt)["N"] = Int64(N)
        # save further attributes
        
        # write bravais lattice vectors
        for i in 1:N
            group_lt["a"*string(i)] = latticeVectors(lt)[i]
        end
    end
    
    # save sites, bonds and unitcell into the file
    saveSites(sites(lt), fn, group_sites, append=true)
    saveBonds(bonds(lt), fn, group_bonds, append=true)
    saveUnitcell(unitcell(lt), fn, group_unitcell, append=true)
    
    # return nothing
    return nothing
end

In [ ]:
function loadLattice(
        ::Type{LT},
        fn :: AbstractString,
        group :: AbstractString = "lattice"
    ) where {SLT<:AbstractSite,BLT<:AbstractBond,ULT<:AbstractUnitcell,LT<:Union{Lattice{SLT,BLT,ULT},Lattice}}
    
    # read the attribute data
    attr_data = h5readattr(fn, group)
    
    # determine the bravais lattice dimension
    N = attr_data["N"]
    
    # determine the site and bond group names
    group_sites    = attr_data["sites"]
    group_bonds    = attr_data["bonds"]
    group_unitcell = attr_data["unitcell"]
    
    # load bravais lattice vectors
    lattice_vectors = Vector{Float64}[]
    for i in 1:N
        push!(lattice_vectors, h5read(fn, group*"/a"*string(i)))
    end
    
    # load the sites
    site_list = loadSites(fn, group_sites)
    # load the bonds
    bond_list = loadBonds(fn, group_bonds)
    # load the unitcell
    unitcell_loaded = loadUnitcell(fn, group_unitcell)
    
    # determine site and bond type
    S = typeof(site_list[1])
    B = typeof(bond_list[1])
    U = typeof(unitcell_loaded)
    
    # return new unitcell
    return newLattice(
        Lattice{S,B,U},
        lattice_vectors,
        site_list,
        bond_list,
        unitcell_loaded
    )
end

function loadLattice(
        fn :: AbstractString,
        group :: AbstractString = "lattice"
    )
    
    return loadLattice(Lattice, fn, group)
end

In [ ]:
uc = getUnitcellHyperoctagon(Int64, Float64)
lt = getLatticeOpen(uc, 10)
saveLattice(lt, "test.h5")
lt

In [ ]:
loadLattice("test.h5")